# Query data from overpassAPI (openstreetmaps)


query now with Python by using the requests package in order to access the Overpass API

https://towardsdatascience.com/loading-data-from-openstreetmap-with-python-and-the-overpass-api-513882a27fd0

## Area[] Berlin 
area["ISO3166-2"="DE-BE"]
https://www.openstreetmap.org/relation/62422

## admin_level=4
admin_level=2 through admin_level=10 allow for different administrative subdivision schemes to be handled in a consistent way by data consumers. The use of a numbering scheme rather than words for the values helps avoid confusion due to different terminology used in different countries
A data consumer looking for municipalities corresponding to "city", "town" or "village" boundaries will find these tagged anywhere from admin_level=4 (e.g. relation Berlin)

In [ ]:
import requests
import json

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-2"="DE-BE"][admin_level=4];
(node['shop'='convenience'](area);
 way['shop'='convenience' ](area);
 rel['shop'='convenience' ](area);
);
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
data = response.json()

In [ ]:
overpass_query

## Plotting 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Collect coords into list
coords = []
for element in data['elements']:
  if element['type'] == 'node':
    lon = element['lon']
    lat = element['lat']
    coords.append((lon, lat))
  elif 'center' in element:
    lon = element['center']['lon']
    lat = element['center']['lat']
    coords.append((lon, lat))# Convert coordinates into numpy array
X = np.array(coords)

plt.plot(X[:, 0], X[:, 1], 'o')
plt.title('Spatkauf in Berlin')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.axis('equal')
plt.show()

# overpy package

In [ ]:
import overpy

api = overpy.Overpass()

r = api.query("""
area["ISO3166-2"="DE-BE"][admin_level=4];
(node["amenity"="biergarten"](area);
 way["amenity"="biergarten"](area);
 rel["amenity"="biergarten"](area);
);
out center;
""")

coords  = []
coords += [(float(node.lon), float(node.lat)) 
           for node in r.nodes]
coords += [(float(way.center_lon), float(way.center_lat)) 
           for way in r.ways]
coords += [(float(rel.center_lon), float(rel.center_lat)) 
           for rel in r.relations]

In [ ]:
r.__dict__

# get boundaries and outline of city

In [ ]:
import overpy
import requests
import json

country_name = 'Germany'
city_name = 'Berlin'

'''example'''
# area[name="Köln"]->.b;
# rel(area.b)[name="Innenstadt"];
# map_to_area -> .a;
# node(area.a)[amenity=pub];
# out meta;

'''not working'''
# study_area = f"""area[name='{country_name}']->.a;
#              rel(area.a)['boundary'='administrative'];
#              map_to_area -> .b;"""
# params = f"""area[name='{city_name}'](area.b);"""

'''not working'''
# study_area = f"""{{{{geocodeArea:{country_name}}}}}->.a;"""
# params = f"""area[name='{city_name}'](area.a);"""
# ['admin_level'='8']

study_area = f"""area['name:en'='{country_name}']->.a"""
params = f"""rel['name'='{city_name}'](area.a)"""

limit = ''
out_type = 'geom'

border_query = f"""{study_area};
                ({params};
                );
                out {limit} {out_type};
                """

In [ ]:
print(border_query)

In [ ]:
api = overpy.Overpass()

overy_request = api.query(border_query)
overy_request

In [ ]:
overy_request.__dict__

In [ ]:
overy_request.relations[0].__dict__

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

overpass_query = f"""
                [out:json][timeout:25];
                {study_area};
                {params};
                out {limit} {out_type};
                """
response = requests.get(overpass_url, 
                            params={'data': overpass_query})
res = response.json()

In [ ]:
id_city = res['elements'][0]['id']

In [ ]:
res['elements'][0]['bounds']

# shapely boundary

In [ ]:
import shapely.geometry as geometry
from shapely.ops import linemerge, unary_union, polygonize


query = f"""[out:json][timeout:25];
rel({id_city});
out body;
>;
out skel qt; """
api = overpy.Overpass()
result = api.query(query)

lss = []  #convert ways to linstrings

for ii_w, way in enumerate(result.ways):
    ls_coords = []

    for node in way.nodes:
        ls_coords.append(
            (node.lon, node.lat))  # create a list of node coordinates

    lss.append(
        geometry.LineString(ls_coords))  # create a LineString from coords

merged = linemerge([*lss])  # merge LineStrings
borders = unary_union(merged)  # linestrings to a MultiLineString
polygons = list(polygonize(borders))
city = geometry.MultiPolygon(polygons)

In [ ]:
id_city

In [ ]:
import numpy as np
import pandas as pd
pd.DataFrame(np.array(result.ways))

In [ ]:
lss = []  #convert ways to linstrings

for ii_w, way in enumerate(result.ways):
    ls_coords = []
    print()
#     for node in way.nodes:
#         ls_coords.append(
#             (node.lon, node.lat))  # create a list of node coordinates

#     lss.append(
#         geometry.LineString(ls_coords))  # create a LineString from coords

## params function

In [ ]:
#not working
# def param_key(keys):
    
#     #params defining the whole key example 'leisure' 
# #     get every type of leisure instalation, as a list returns several keys
    
#     string_keys = ''
#     for k in keys: 
#         string_keys += f"""node[{k}](area);"""
#     return string_keys

# def param_key_value(key, value):
    
#     #params defining the key and value, example 'leisure' 'parks' will only get parks
    
#     string_key = f"""\
#     node['{key}'='{value}'](area);
#     way['{key}'='{value}'](area);
#     rel['{key}'='{value}'](area);
#     """
#     return string_key

## imports

In [ ]:
import requests
import json

In [ ]:
#style working properly
# osm_keys_ok = """node["amenity"="pub"](area.berlin);
#             node["amenity"="bar"](area.berlin); 
#             """

## query function

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

# study_area = 'area["ISO3166-2"="DE-BE"]->.berlin'
study_area = f"""area(62422)->.bound"""

out_type = 'body qt'

keys_values_osm = {'amenity':['bbq','drinking_water']}


def param_nodes(keys):
    '''converts the dict into a string, returns a str'''
    osm_keys = ''
    for k,val in keys.items():
        for v in val:
            osm_keys += f"""nwr['{k}'='{v}'](area.bound);"""
    return osm_keys
    
def param_ways(keys):
    '''converts the dict into a string, returns a str'''
    osm_keys = ''
    for k,val in keys.items():
        for v in val:
            osm_keys += f"""way['{k}'='{v}'](area.bound);"""
    return osm_keys

def param_areas(keys):
    '''converts the dict into a string, returns a str'''
    osm_keys = ''
    for k,val in keys.items():
        for v in val:
            osm_keys += f"""area['{k}'='{v}'](area.bound);"""
    return osm_keys


def query_params_osm(keys, feat, limit=''):
    '''Adding keys and values as a dictionary, example: keys_values_osm = {'amenity':['bbq','cafe']}
    several values can added to a same key as a list, returns a dict
    feat = nodes, ways or areas (geometry type)
    limit = number (optional query limit)'''
    
    if feat == 'ways':
        params = param_ways(keys)
        out_type = 'geom'
    if feat == 'areas':
        params = param_areas(keys)
        out_type = 'geom'
    if feat == 'nodes':
        params = param_nodes(keys)
        out_type = 'center'
        
    overpass_query = f"""[out:json][timeout:25];{study_area};({params});
                        out {limit} {out_type};
                    """
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    return response.json(),

In [ ]:
keys_values_osm = {'amenity':['bbq']}


In [ ]:
from geopy.geocoders import Nominatim
import overpy

city_name = "berlin"

# Geocoding request via Nominatim
geolocator = Nominatim(user_agent="city_compare")
geo_results = geolocator.geocode(city_name, exactly_one=False, limit=3)

# Searching for relation in result set
for r in geo_results:
    print(r.address, r.raw.get("osm_type"))
    if r.raw.get("osm_type") == "relation":
        city = r
        break

# Calculating area id
area_id = int(city.raw.get("osm_id")) + 3600000000

# Excecuting overpass call
api = overpy.Overpass()
result = api.query(f"""
    area(%s)->.bound;
    (
      {param_nodes(keys_values_osm)}
    );
    out body;
    """ % area_id)

# Printing no. of pubs in nodes and ways
print("Amenities in nodes: %d" % len(result.nodes))
print("Amenities in ways: %d" % len(result.ways))
print("Amenities in areas: %d" % len(result.areas))

In [ ]:
result.nodes

In [ ]:
keys_values_osm

In [ ]:
param_nodes(keys_values_osm)

In [ ]:
data = query_params_osm(keys_values_osm, feat='nodes')

In [ ]:
data

In [ ]:
len(data['elements'])

# Params for Mobility

## Get dict for public transport

nodes

In [ ]:
public_transport = {'amenity':['bus_station'],
                    'bus_bay':'',
                   'highway':['bus_stop','platform'],
                   'public_transport':['stop_position','platform','station','stop_area'],
                   'railway':['station','tram_stop', 'subway_entrance']}

bike_infraestructure =  {'amenity':['bicycle_parking', 'bicycle_repair_station', 'bicycle_rental']}

In [ ]:
param_nodes(public_transport)

In [ ]:
query_publictransport = query_params_osm(public_transport, 'nodes')
query_bikeinfraestructure = query_params_osm(bike_infraestructure, 'nodes')

## Get dict for Cyclelanes and pedestrians

Ways queries

In [ ]:
cycle_paths = {'bicycle':['designated']}
# 'highway':['cycleway']

In [ ]:
param_ways(cycle_paths)

In [ ]:
data_cycle = query_params_osm(cycle_paths, feat='ways')

In [ ]:
len(data_cycle['elements'])

# Params for Social Life

nodes

In [ ]:
eating = {'amenity':['cafe','restaurant', 'food_court', 'ice_cream']}

night_life = { 'amenity':['bar','pub','biergarten', 'nightclub', 'swingerclub', 'casino']}

culture = {'amenity':['social_centre','theatre','public_bookcase',
                      'fountain', 'events_venue', 'community_centre', 'cinema', 
                      'arts_centre', 'conference_centre']}

community = {'office': ['association','charity', 'coworking', 
                        'educational_institution', 'employment_agency', 'foundation', 
                        'ngo', ; 'political_party', 'research']}

# Params for Activities


nodes

In [ ]:
health_care = {'amenity':['baby_hatch','clinic','dentist',
                      'doctors', 'hospital', 'nursing_home', 'pharmacy', 
                      'social_facility', 'veterinary']}

public_service = {'amenity':['courthouse','fire_station','police',
                      'post_box', 'post_office', 'townhall']}

education = {'amenity':['college','kindergarten','language_school',
                      'library', 'music_school', 'school', 'university']}

economic = {'amenity':['atm', 'bank', 'bureau_de_change']}

#  Params for Comfort

## Node Queries

In [ ]:
comfort_spots = {'amenity':['bbq','bench','dog_toilet',
                      '	drinking_water', 'give_box', 'shelter', 'shower', 
                      'toilets', 'water_point', 'watering_place']}

leisure_spots = {'leisure':['bandstand','bird_hide','dog_park',
                      'firepit', 'swimming_pool', 'stadium', 'sports_centre', 
                      'pitch', 'picnic_table', 'fitness_centre'],
                 'historic':''}

trees = []


## Area querys

In [ ]:
leirsure_areas = {'leisure':['park','garden','swimming_area',
                      'playground', 'nature_reserve', 'marina'],
                 'landuse':['forest']}

# test of geometries query 

In [ ]:
test = {'amenity':['bbq']}

In [ ]:
leisure = query_params_osm(leirsure_areas, 'areas',10)

In [ ]:
len(leisure['elements'])

In [ ]:
leisure

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def osm_json_to_DF(response):
    response_elements = response['elements']
    elements_tmp = []

    for element in response_elements:
        elements_tmp.append(element['geometry'])

    return pd.DataFrame({'Coords':elements_tmp})

In [ ]:
df1 = osm_json_to_DF(leisure)

In [ ]:
df1

## converting to csv

In [ ]:
df1.to_csv('../livablestreets/data/areas_test.csv', index=False)

# quering with a pattern DF

In [251]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from livablestreets.osm_query import query_params_osm
from livablestreets.query_names_detailed import master_query, master_query_complex, master_query_negative

import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [256]:
query_df = master_query_negative()

In [257]:
query_df

,query_string,name,distance,geomtype,jsontype,shapelytype,category
street_motorway,{'highway': ['motorway']},street_motorway,500,Way,Line,lineString,negative
street_primary,{'highway': ['primary']},street_primary,250,Way,Line,lineString,negative
street_secondary,{'highway': ['secondary']},street_secondary,100,Way,Line,lineString,negative
street_terteary,{'highway': ['terteary']},street_terteary,50,Way,Line,lineString,negative
railway,"{'railway': ['light_rail'], 'bridge': ['viaduc...",railway,50,Way,Line,lineString,negative


In [254]:
query_df = query_df.iloc[2:3]
query_df

,query_string,name,distance,geomtype,jsontype,shapelytype,category
street_secondary,{'highway': ['secondary']},street_secondary,100,Way,Line,lineString,negative


In [255]:
ne =df[df['geometry'].notna()]

KeyError: 'geometry'

In [189]:
ne.head()

,type,id,lat,lon,tags,bounds,nodes,geometry
114,way,42465643,NaN,NaN,"{'highway': 'footway', 'lit': 'yes', 'surface'...","{'minlat': 49.6927106, 'minlon': 6.5770306, 'm...","[530193230, 530193219, 530193221, 6379328970]","[{'lat': 49.693162, 'lon': 6.5771262}, {'lat':..."
115,way,56132342,NaN,NaN,{'highway': 'footway'},"{'minlat': 43.6108665, 'minlon': 3.483429, 'ma...","[704411344, 7099941847, 7099941846, 7099941845...","[{'lat': 43.6108665, 'lon': 3.483429}, {'lat':..."
116,way,197914145,NaN,NaN,{'highway': 'footway'},"{'minlat': 43.6059594, 'minlon': 3.4949988, 'm...","[705690069, 2081125961, 2081125973, 2081125975]","[{'lat': 43.6059594, 'lon': 3.4950204}, {'lat'..."
117,way,198468259,NaN,NaN,"{'bridge': 'yes', 'highway': 'footway', 'layer...","{'minlat': 43.6012006, 'minlon': 3.4833954, 'm...","[2085948036, 2085948031]","[{'lat': 43.6012077, 'lon': 3.4833954}, {'lat'..."
118,way,205948986,NaN,NaN,{'highway': 'footway'},"{'minlat': 43.6011588, 'minlon': 3.4836402, 'm...","[2085948031, 2085948021, 2085948017, 208594801...","[{'lat': 43.6012006, 'lon': 3.4836402}, {'lat'..."


In [190]:
xss = ne['geometry']
flat_list = [x for xs in xss for x in xs]

In [191]:
flat_list

[{'lat': 49.693162, 'lon': 6.5771262},
 {'lat': 49.6930275, 'lon': 6.577048},
 {'lat': 49.6929167, 'lon': 6.5770306},
 {'lat': 49.6927106, 'lon': 6.5772754},
 {'lat': 43.6108665, 'lon': 3.483429},
 {'lat': 43.6109833, 'lon': 3.4834788},
 {'lat': 43.6109693, 'lon': 3.4835605},
 {'lat': 43.6110462, 'lon': 3.4835873},
 {'lat': 43.6110724, 'lon': 3.4835383},
 {'lat': 43.611113, 'lon': 3.4835434},
 {'lat': 43.6112593, 'lon': 3.4836153},
 {'lat': 43.6112652, 'lon': 3.4836826},
 {'lat': 43.6113943, 'lon': 3.4837362},
 {'lat': 43.6059594, 'lon': 3.4950204},
 {'lat': 43.6060568, 'lon': 3.4950201},
 {'lat': 43.6061772, 'lon': 3.4950175},
 {'lat': 43.6062606, 'lon': 3.4949988},
 {'lat': 43.6012077, 'lon': 3.4833954},
 {'lat': 43.6012006, 'lon': 3.4836402},
 {'lat': 43.6012006, 'lon': 3.4836402},
 {'lat': 43.6011839, 'lon': 3.4836868},
 {'lat': 43.6011683, 'lon': 3.4837191},
 {'lat': 43.6011647, 'lon': 3.4837895},
 {'lat': 43.6011588, 'lon': 3.483872},
 {'lat': 43.6012232, 'lon': 3.4838815},
 {'la

In [192]:
df_new_querie = pd.DataFrame(flat_list)[['lat', 'lon']]
df_new_querie['coor'] = list(zip(df_new_querie.lat, df_new_querie.lon))

In [193]:
df_new_querie

,lat,lon,coor
0,49.693162,6.577126,"(49.693162, 6.5771262)"
1,49.693027,6.577048,"(49.6930275, 6.577048)"
2,49.692917,6.577031,"(49.6929167, 6.5770306)"
3,49.692711,6.577275,"(49.6927106, 6.5772754)"
4,43.610867,3.483429,"(43.6108665, 3.483429)"
...,...,...,...
115,43.611434,3.483579,"(43.6114337, 3.4835793)"
116,43.602533,3.491026,"(43.6025326, 3.4910262)"
117,43.602516,3.491045,"(43.6025158, 3.4910454)"
118,43.611266,3.482800,"(43.6112656, 3.4828001)"


In [264]:
def get_csv(city):
    
    #query_df = df.iloc[2:]
    for index, row in query_df.iterrows():
        filter_name = index
        string = row['query_string']
        category = row['category']
        geomtype = row['geomtype']
#         cwd = os.getcwd()
        
#         if not os.path.exists(f'{cwd}/data/{city.lower()}/Features/{category}_{filter_name}.csv'):

        if geomtype != 'Node':
            print(f'getting {filter_name} as ways')
            new_querie = query_params_osm(location = city.capitalize(), keys = string, features = 'ways')
#             print(new_querie['elements'])
            
            if new_querie['elements']:
                df_new_querie = pd.DataFrame(new_querie['elements'])
                print(f'------------------------------------------------------------>')

                ne = df_new_querie[df_new_querie['geometry'].notna()]
                xss = ne['geometry']
                flat_list = [x for xs in xss for x in xs]
                df_new_querie = pd.DataFrame(flat_list)[['lat', 'lon']]
                df_new_querie['coor'] = list(zip(df_new_querie.lat, df_new_querie.lon))
                print(f'waiting time 60seconds')
                time.sleep(3)


        if geomtype == 'Node':
            print(f'getting {filter_name} as nodes')
            new_querie = query_params_osm(location = city.capitalize(), keys = string, features = 'nodes')
            if new_querie['elements']:
                print(f'------------------------> {new_querie}')

                df_new_querie = pd.DataFrame(new_querie['elements'])[['lat', 'lon']]
                df_new_querie['coor'] = list(zip(df_new_querie.lat, df_new_querie.lon))
            
        print(df_new_querie.head(2))

#         
#         df_new_querie.to_csv(f'{cwd}/data/{city.lower()}/Features/{category}_{filter_name}.csv', index=False)

In [265]:
query_df

,query_string,name,distance,geomtype,jsontype,shapelytype,category
street_motorway,{'highway': ['motorway']},street_motorway,500,Way,Line,lineString,negative
street_primary,{'highway': ['primary']},street_primary,250,Way,Line,lineString,negative
street_secondary,{'highway': ['secondary']},street_secondary,100,Way,Line,lineString,negative
street_terteary,{'highway': ['terteary']},street_terteary,50,Way,Line,lineString,negative
railway,"{'railway': ['light_rail'], 'bridge': ['viaduc...",railway,50,Way,Line,lineString,negative


In [266]:
get_csv('Canet')

getting street_motorway as ways
------------------------------------------------------------>
waiting time 60seconds
         lat       lon                     coor
0  43.585179  3.465471  (43.5851792, 3.4654706)
1  43.586440  3.465352  (43.5864398, 3.4653522)
getting street_primary as ways
------------------------------------------------------------>
waiting time 60seconds
         lat       lon                     coor
0  43.597101  3.493232   (43.597101, 3.4932322)
1  43.598088  3.492721  (43.5980877, 3.4927211)
getting street_secondary as ways


JSONDecodeError: [Errno Expecting value] <?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN"
    "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">
<head>
  <meta http-equiv="content-type" content="text/html; charset=utf-8" lang="en"/>
  <title>OSM3S Response</title>
</head>
<body>

<p>The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.</p>
<p><strong style="color:#FF0000">Error</strong>: runtime error: open64: 0 Success /osm3s_v0.7.58_osm_base Dispatcher_Client::request_read_and_idx::timeout. The server is probably too busy to handle your request. </p>

</body>
</html>
: 0

In [9]:
location = 'cit'
print(f'{{{{geocodeArea:{location}}}}}->.city')

{{geocodeArea:cit}}->.city
